In [1]:
from SARL_Env_v3 import MazeEnv

from stable_baselines3 import A2C, PPO
import optuna

In [2]:
# from JavalessClient import Client

In [3]:
# c = Client("manager", "127.0.0.1", 9888, True)

In [4]:
# c.login_manager()

In [5]:
from gym.spaces import Discrete, Box, Tuple, MultiBinary, MultiDiscrete, Dict
import numpy as np

In [6]:
menv = MazeEnv()#.to(device='cuda')

In [7]:
steps = 1000000

In [8]:
def objective(trial):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 0.01, log=True)
    n_steps = trial.suggest_int('n_steps', 1, 20)
    gamma = trial.suggest_float("gamma", 0, 1)
    gae_lambda = trial.suggest_float("gae_lambda", 0, 1)
    ent_coef = trial.suggest_float("ent_coef", 0, 2)
    vf_coef = trial.suggest_float("vf_coef", 0, 1)
    max_grad_norm = trial.suggest_float("max_grad_norm", 1e-8, 1)
    normalize_advantage = trial.suggest_categorical("normalize_advantage", [True, False])

    # Create A2C agent with current hyperparameters
    env = MazeEnv()
    model = A2C('MultiInputPolicy',
                env,
                learning_rate=learning_rate,
                n_steps=n_steps,
                gamma=gamma,
                gae_lambda=gae_lambda,
                ent_coef=ent_coef,
                vf_coef=vf_coef,
                max_grad_norm=max_grad_norm,
                normalize_advantage=normalize_advantage,
                device="cuda"
               )

    # Train agent and evaluate performance
    model.learn(total_timesteps=1000)
    eval_reward = evaluate_agent(model)

    # Report the evaluation reward as the result of this trial
    return eval_reward

In [9]:
# Define a function to evaluate the performance of the trained agent
def evaluate_agent(model):
    # Evaluate the agent on the environment for a fixed number of episodes
    env = MazeEnv()
    obs = env.reset()
    total_reward = 0
    for i in range(10):
        done = False
        while not done:
            action, _ = model.predict(obs)
            obs, reward, done, trunc, info = env.step(action)
            total_reward += reward
    avg_reward = total_reward / 10
    return avg_reward

In [10]:
# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Retrieve the best hyperparameters and train the agent for a longer period of time
best_learning_rate = study.best_params['learning_rate']
best_n_steps = study.best_params['n_steps']
best_gamma = study.best_params['gamma']
best_gae_lambda = study.best_params['gae_lambda']
best_ent_coef = study.best_params['ent_coef']
best_vf_coef = study.best_params['vf_coef']
best_max_grad_norm = study.best_params['max_grad_norm']

[I 2023-03-20 17:19:53,350] A new study created in memory with name: no-name-3f5a9c1b-7727-4bb0-a4e6-eb9dd396b8db


nice


[W 2023-03-20 17:20:42,255] Trial 0 failed with parameters: {'learning_rate': 0.0003318955629804542, 'n_steps': 3, 'gamma': 0.896185730460487, 'gae_lambda': 0.3001191941018235, 'ent_coef': 0.08695591791521617, 'vf_coef': 0.7212094100847236, 'max_grad_norm': 0.43822250885986397, 'normalize_advantage': True} because of the following error: IndexError('only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices').
Traceback (most recent call last):
  File "C:\Users\aylam\AppData\Roaming\Python\Python39\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\aylam\AppData\Local\Temp\ipykernel_13244\4034818103.py", line 29, in objective
    eval_reward = evaluate_agent(model)
  File "C:\Users\aylam\AppData\Local\Temp\ipykernel_13244\2680438499.py", line 14, in evaluate_agent
    action, _ = model.predict(obs)
  File "C:\Users\aylam\AppData\Roaming\Python\Python39\site-packa

hey 1
hey 2


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
from numpy import array
j=({'player_pos': array([0, 0]), 'next_treasure_pos': array([0, 0]), 'reachable_positions': array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0]], dtype=int)}, {})

In [ ]:
j[1]

In [ ]:
pl

In [ ]:
path = 'Training/Logs'
model = A2C('MultiInputPolicy',
            menv,
            verbose=1,
            tensorboard_log=path,
            learning_rate=best_learning_rate
            n_steps=best_n_steps,
            gamma=best_gamma,
            gae_lambda=best_gae_lambda,
            ent_coef=best_ent_coef,
            vf_coef=best_vf_coef,
            max_grad_norm=best_max_grad_norm
            device="cuda",
            normalize_advantage=best_normalize_advantage,
           )

In [ ]:
model.learn(total_timesteps=steps)

In [ ]:
a2c_path = "Training/Saved Models/PPO_CNN_"+str(steps)+"_Mazenet_Model"
model.save(a2c_path)

In [ ]:
k

In [ ]:
obs = menv.reset()
done = False  
total_reward   = 0
while not done:
    print(menv.action_space.sample())
    obs, reward,  done, info =  menv.step(menv.action_space.sample())
    total_reward  += reward
print('Total Reward for episode {} is {}'.format(episode, total_reward))

In [ ]:
from stable_baselines3.common.env_checker import check_env

env = MazeEnv()
# It will check your custom environment and output additional warnings if needed
check_env(env)